In [46]:
import psycopg2
import io
import csv

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="Lucas123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

queries = []

## Query Examples

Below are 5 non-trivial queries, which demonstrate the usability of the project for complex analysis of data from the São Paulo Municipal Public School Network.

### 1. Student Distribution by Race and Country of Origin in Each District

#### Explanation:

This query counts the number of distinct students (`id_aluno`) grouped by their race (`desc_raca_cor`) and country of origin (`desc_pais_nasc`). The results are ordered by the total number of students in descending order.

#### Real-World Use:

- Diversity Analysis: Assesses the racial and ethnic diversity within schools.

- Policy Development: Informs policies aimed at promoting inclusivity and equity.

- Community Engagement: Helps in understanding the demographic composition of school communities.

In [47]:
queries.append(("""
SELECT r.nome_distrito AS district, a.raca_cor AS race, a.pais_nasc AS birth_country, COUNT(DISTINCT a.id_aluno) AS total_students
FROM Aluno a
INNER JOIN (SELECT t.nome_turma, e.nome_distrito
			FROM Turma t
            INNER JOIN Escola e ON t.nome_esc = e.nome_esc) r 
            ON a.nome_turma = r.nome_turma
GROUP BY r.nome_distrito, a.raca_cor, a.pais_nasc
ORDER BY total_students DESC;
""", "query1.csv"))

### 2. Students with Disabilities by Type in Each School

#### Explanation:

This query counts the number of distinct students (`id_aluno`) with non-null disability types (`nee`). The results are grouped by disability type and ordered by the total number of students in descending order.

#### Real-World Use:

- Special Education Planning: Assists in allocating resources for students with disabilities.

- Policy Formulation: Informs policies to support inclusive education.

- Needs Assessment: Identifies areas requiring specialized services.

In [48]:
queries.append(("""
SELECT r.nome_distrito AS district, a.nee, COUNT(DISTINCT a.id_aluno) AS total_students
FROM Aluno a
INNER JOIN (SELECT t.nome_turma, e.nome_distrito
            FROM Turma t
            INNER JOIN Escola e ON t.nome_esc = e.nome_esc) r 
            ON a.nome_turma = r.nome_turma
WHERE nee IS NOT NULL
GROUP BY r.nome_distrito, a.nee
ORDER BY total_students DESC;
""", "query2.csv"))

### 3. Open spaces in classes per district

#### Explanation:

This query calculates, by each district, what is the number of the vacant places in classes (total number of places - number of enrolled students)

#### Real-World Use:

- Resource Allocation: Identifying districts with schools that have available capacity, aiding in decisions about where to allocate resources or open new classes.

- Policy Planning: Assisting policymakers in understanding enrollment patterns and making informed decisions about school expansions or consolidations.

- Community Engagement: Providing transparency to the community about school capacities and potential for accommodating additional students.

In [49]:
queries.append(("""
SELECT e.nome_distrito AS district, SUM(t.vagas - t.matriculados) AS open_spaces
FROM Turma t
INNER JOIN  Escola e ON t.nome_esc = e.nome_esc
GROUP BY e.nome_distrito
ORDER BY open_spaces DESC;
""", "query3.csv"))

### 4. Different Ambients per School Type

#### Explanation:

This query identifies the different types of environments (ambients) used in each school:

- Selecting the school name (`nome_esc`) and the environment code (`cod_amb`).

- Counting the distinct environment codes for each school to determine the variety of environments.

- Grouping the results by school and environment code, and ordering them by the number of distinct environments in descending order.

#### Real-World Use:

- Infrastructure Planning: Assessing whether schools are utilizing their facilities effectively or if there is a need for additional specialized spaces.

- Curriculum Development: Informing decisions about the types of environments needed to support different teaching methods and learning activities.

- Facility Management: Assisting in the management and maintenance of various school environments to ensure they meet educational needs.

In [50]:
queries.append(("""
SELECT e.nome_esc as school, a.desc_amb AS ambient, COUNT(DISTINCT a.cod_amb) AS total_ambients
FROM Ambiente a
INNER JOIN Escola e ON a.nome_esc = e.nome_esc
GROUP BY e.nome_esc, a.desc_amb
ORDER BY total_ambients DESC, school;
""", "query4.csv"))

### 5. Average Age of Students by School

#### Explanation:

This query calculates the average age of students in each school by subtracting the birth year (`nascimento`) from the current year. The results are grouped by school name (`nome_esc`) and ordered by average age in descending order.

#### Real-World Use:

- Curriculum Design: Assists in tailoring educational content to age groups.

- Age Distribution Analysis: Provides insights into the age demographics of students.

- Policy Planning: Informs decisions on age-appropriate educational strategies.

In [ ]:
queries.append(("""
SELECT r.nome_esc AS escola, ROUND(AVG(EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM a.nascimento)), 0) AS average_age
FROM Aluno a
INNER JOIN (SELECT t.nome_turma, e.nome_esc
            FROM Turma t
            INNER JOIN Escola e ON t.nome_esc = e.nome_esc) r 
            ON a.nome_turma = r.nome_turma
			WHERE a.nascimento IS NOT NULL
GROUP BY r.nome_esc 
ORDER BY average_age DESC;
""", "query5.csv"))

In [52]:
for query, filename in queries:
    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()
        with open(filename, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([desc[0] for desc in cur.description])  # Write header
            writer.writerows(rows)

In [53]:
conn.commit()
cur.close()
conn.close()